In [4]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

# Environment 

In [5]:
#Select Variable 
var = "solidprcptot"
#NorESM1-M_rcp45_solidprcptot_monthly

# Run all from here ############

In [6]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

In [7]:
variable85="rcp85_"+var+"_annual"
variable45="rcp45_"+var+"_annual"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_solidprcptot_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_

# FOR RCP 45

## Subsetting arrays by periods of time and percentiles with Xclim

In [8]:
dsEns45= ens.create_ensemble(files45)
dsEns45

<xarray.Dataset>
Dimensions:       (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lon           (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * lat           (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time          (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * realization   (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    solidprcptot  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [9]:
# Mapping data - 30 year means
tmp145 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time145 = tmp145.time[0::window]

ds30yavg45 = tmp145.coarsen(time=window).mean()
ds30yavg45['time'] = time145
perc30yavg45 = ens.ensemble_percentiles(ds30yavg45)
perc30yavg45 # entire grid

# Mapping data

<xarray.Dataset>
Dimensions:           (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * lon               (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * lat               (lat) float32 66.62331 66.53998 ... 40.12437 40.04104
  * time              (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * realization       (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    solidprcptot_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    solidprcptot_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    solidprcptot_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [18]:
df45 = perc30yavg45.drop('realization').to_dataframe().dropna()

df45

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



solidprcptot_p10  solidprcptot_p50  \
lat       lon        time                                             
66.623306 -89.045212 1981-01-01        134.469803        144.152130   
                     2011-01-01        136.434738        141.374817   
                     2041-01-01        137.142792        148.176514   
                     2071-01-01        141.461853        148.134995   
          -88.961884 1981-01-01        132.718475        141.713654   
...                                           ...               ...   
40.041039 -74.129143 2071-01-01         13.079515         25.774853   
          -74.045807 1981-01-01         39.821648         54.053120   
                     2011-01-01         27.579977         46.238033   
                     2041-01-01         21.263029         33.893375   
                     2071-01-01         13.295259         25.998478   

                                 solidprcptot_p90  
lat       lon        time                          
66.623306 -89.045212 1981-01-01        149.838272  
                     2011-01-01        152.103210  
                     2041-01-01        156.438156  
                     2071-01-01        164.467957  
          -88.961884 1981-01-01        147.689972  
...                                           ...  
40.041039 -74.129143 2071-01-01         35.280640  
          -74.045807 1981-01-01         67.982697  
                     2011-01-01         53.271015  
                     2041-01-01         43.802750  
                     2071-01-01         33.471703  

[304824 rows x 3 columns]

## Transform in Celsius and round

In [19]:
df45C = df45.copy()
df45C[var+"_p10"] = round(df45C[var+"_p10"]/10, 2)
df45C[var+"_p50"] = round(df45C[var+"_p50"]/10, 2)
df45C[var+"_p90"] = round(df45C[var+"_p90"]/10, 2)

df45C

solidprcptot_p10  solidprcptot_p50  \
lat       lon        time                                             
66.623306 -89.045212 1981-01-01             13.45             14.42   
                     2011-01-01             13.64             14.14   
                     2041-01-01             13.71             14.82   
                     2071-01-01             14.15             14.81   
          -88.961884 1981-01-01             13.27             14.17   
...                                           ...               ...   
40.041039 -74.129143 2071-01-01              1.31              2.58   
          -74.045807 1981-01-01              3.98              5.41   
                     2011-01-01              2.76              4.62   
                     2041-01-01              2.13              3.39   
                     2071-01-01              1.33              2.60   

                                 solidprcptot_p90  
lat       lon        time                          
66.623306 -89.045212 1981-01-01             14.98  
                     2011-01-01             15.21  
                     2041-01-01             15.64  
                     2071-01-01             16.45  
          -88.961884 1981-01-01             14.77  
...                                           ...  
40.041039 -74.129143 2071-01-01              3.53  
          -74.045807 1981-01-01              6.80  
                     2011-01-01              5.33  
                     2041-01-01              4.38  
                     2071-01-01              3.35  

[304824 rows x 3 columns]

## Eliminate 2011, Pivot Table and Create new columns 

In [20]:
Region1i45 = df45C.reset_index()
print (Region1i45)
Region1i245 = Region1i45[Region1i45.time.dt.year!= 2011]
print ("2011 eliminated")
df45f = Region1i245.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df45f.columns = [year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df45f

              lat        lon       time  solidprcptot_p10  solidprcptot_p50  \
0       66.623306 -89.045212 1981-01-01             13.45             14.42   
1       66.623306 -89.045212 2011-01-01             13.64             14.14   
2       66.623306 -89.045212 2041-01-01             13.71             14.82   
3       66.623306 -89.045212 2071-01-01             14.15             14.81   
4       66.623306 -88.961884 1981-01-01             13.27             14.17   
...           ...        ...        ...               ...               ...   
304819  40.041039 -74.129143 2071-01-01              1.31              2.58   
304820  40.041039 -74.045807 1981-01-01              3.98              5.41   
304821  40.041039 -74.045807 2011-01-01              2.76              4.62   
304822  40.041039 -74.045807 2041-01-01              2.13              3.39   
304823  40.041039 -74.045807 2071-01-01              1.33              2.60   

        solidprcptot_p90  
0                  14.98

hist_rcp45_p10  t2050_rcp45_p10  t2080_rcp45_p10  \
lat       lon                                                            
40.041039 -89.045212            5.78             3.29             3.32   
          -88.961884            5.93             3.35             3.36   
          -88.878548            6.04             3.37             3.39   
          -88.795219            6.04             3.33             3.36   
          -88.711891            6.06             3.27             3.30   
...                              ...              ...              ...   
66.623306 -61.629639           32.37            35.02            34.88   
          -61.546310           33.55            36.79            36.48   
          -61.462978           32.99            35.69            35.50   
          -61.379650           32.55            35.15            35.20   
          -61.296322           29.51            31.37            30.84   

                      hist_rcp45_p50  t2050_rcp45_p50  t2080_rcp45_p50  \
lat       lon                                                            
40.041039 -89.045212            6.94             5.44             4.68   
          -88.961884            7.11             5.56             4.72   
          -88.878548            7.12             5.64             4.76   
          -88.795219            7.14             5.66             4.78   
          -88.711891            7.11             5.67             4.68   
...                              ...              ...              ...   
66.623306 -61.629639           34.00            36.67            38.81   
          -61.546310           35.29            38.72            40.47   
          -61.462978           34.37            37.45            39.31   
          -61.379650           33.91            37.15            38.93   
          -61.296322           30.67            33.28            34.90   

                      hist_rcp45_p90  t2050_rcp45_p90  t2080_rcp45_p90  
lat       lon                                                           
40.041039 -89.045212            8.95             7.10             6.45  
          -88.961884            9.16             7.28             6.55  
          -88.878548            9.24             7.29             6.63  
          -88.795219            9.16             7.26             6.65  
          -88.711891            9.19             7.17             6.65  
...                              ...              ...              ...  
66.623306 -61.629639           36.14            41.11            42.72  
          -61.546310           37.93            43.01            44.13  
          -61.462978           36.70            41.86            43.32  
          -61.379650           36.12            41.25            42.71  
          -61.296322           32.65            37.24            38.55  

[76206 rows x 9 columns]

# FOR RCP 85

## Subsetting arrays by periods of time and percentiles with Xclim

In [21]:
dsEns85= ens.create_ensemble(files85)
dsEns85

<xarray.Dataset>
Dimensions:       (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lon           (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * lat           (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time          (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * realization   (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    solidprcptot  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [22]:
# Mapping data - 30 year means
tmp185 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time185 = tmp185.time[0::window]

ds30yavg85 = tmp185.coarsen(time=window).mean()
ds30yavg85['time'] = time185
perc30yavg85 = ens.ensemble_percentiles(ds30yavg85)
perc30yavg85 # entire grid

<xarray.Dataset>
Dimensions:           (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * lon               (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * lat               (lat) float32 66.62331 66.53998 ... 40.12437 40.04104
  * time              (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * realization       (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    solidprcptot_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    solidprcptot_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    solidprcptot_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [23]:
%time df85 = perc30yavg85.drop('realization').to_dataframe().dropna()
df85

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 1min 15s, sys: 9min 20s, total: 10min 36s
Wall time: 9.08 s


solidprcptot_p10  solidprcptot_p50  \
lat       lon        time                                             
66.623306 -89.045212 1981-01-01        134.306290        141.427979   
                     2011-01-01        140.742996        145.308243   
                     2041-01-01        130.448059        150.090363   
                     2071-01-01        132.001740        147.813721   
          -88.961884 1981-01-01        132.377487        138.513962   
...                                           ...               ...   
40.041039 -74.129143 2071-01-01          5.324550         11.170806   
          -74.045807 1981-01-01         41.445347         56.142326   
                     2011-01-01         29.839640         39.268230   
                     2041-01-01         15.316210         26.317339   
                     2071-01-01          5.466074         10.986025   

                                 solidprcptot_p90  
lat       lon        time                          
66.623306 -89.045212 1981-01-01        149.385483  
                     2011-01-01        151.985992  
                     2041-01-01        156.690598  
                     2071-01-01        169.476517  
          -88.961884 1981-01-01        146.892776  
...                                           ...  
40.041039 -74.129143 2071-01-01         20.912359  
          -74.045807 1981-01-01         67.438210  
                     2011-01-01         47.335026  
                     2041-01-01         33.731472  
                     2071-01-01         20.836275  

[304824 rows x 3 columns]

## Transform in Celsius and round

In [24]:
df85C = df85.copy()
df85C[var+"_p10"] = round(df85C[var+"_p10"]/10, 2)
df85C[var+"_p50"] = round(df85C[var+"_p50"]/10, 2)
df85C[var+"_p90"] = round(df85C[var+"_p90"]/10, 2)
df85C

solidprcptot_p10  solidprcptot_p50  \
lat       lon        time                                             
66.623306 -89.045212 1981-01-01             13.43             14.14   
                     2011-01-01             14.07             14.53   
                     2041-01-01             13.04             15.01   
                     2071-01-01             13.20             14.78   
          -88.961884 1981-01-01             13.24             13.85   
...                                           ...               ...   
40.041039 -74.129143 2071-01-01              0.53              1.12   
          -74.045807 1981-01-01              4.14              5.61   
                     2011-01-01              2.98              3.93   
                     2041-01-01              1.53              2.63   
                     2071-01-01              0.55              1.10   

                                 solidprcptot_p90  
lat       lon        time                          
66.623306 -89.045212 1981-01-01             14.94  
                     2011-01-01             15.20  
                     2041-01-01             15.67  
                     2071-01-01             16.95  
          -88.961884 1981-01-01             14.69  
...                                           ...  
40.041039 -74.129143 2071-01-01              2.09  
          -74.045807 1981-01-01              6.74  
                     2011-01-01              4.73  
                     2041-01-01              3.37  
                     2071-01-01              2.08  

[304824 rows x 3 columns]

## Eliminate 2011, Pivot Table and Create new columns 

In [25]:
Region1i85 = df85C.reset_index()
print (Region1i85)
Region1i285 = Region1i85[Region1i85.time.dt.year!= 2011]
print ("2011 eliminated")
df85f = Region1i285.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df85f.columns = [year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df85f

              lat        lon       time  solidprcptot_p10  solidprcptot_p50  \
0       66.623306 -89.045212 1981-01-01             13.43             14.14   
1       66.623306 -89.045212 2011-01-01             14.07             14.53   
2       66.623306 -89.045212 2041-01-01             13.04             15.01   
3       66.623306 -89.045212 2071-01-01             13.20             14.78   
4       66.623306 -88.961884 1981-01-01             13.24             13.85   
...           ...        ...        ...               ...               ...   
304819  40.041039 -74.129143 2071-01-01              0.53              1.12   
304820  40.041039 -74.045807 1981-01-01              4.14              5.61   
304821  40.041039 -74.045807 2011-01-01              2.98              3.93   
304822  40.041039 -74.045807 2041-01-01              1.53              2.63   
304823  40.041039 -74.045807 2071-01-01              0.55              1.10   

        solidprcptot_p90  
0                  14.94

hist_rcp85_p10  t2050_rcp85_p10  t2080_rcp85_p10  \
lat       lon                                                            
40.041039 -89.045212            6.03             2.77             1.60   
          -88.961884            6.23             2.85             1.61   
          -88.878548            6.34             2.90             1.60   
          -88.795219            6.44             2.91             1.66   
          -88.711891            6.37             2.85             1.59   
...                              ...              ...              ...   
66.623306 -61.629639           31.65            32.11            29.49   
          -61.546310           33.08            34.50            31.98   
          -61.462978           32.10            33.64            31.19   
          -61.379650           31.69            33.31            30.88   
          -61.296322           28.54            29.48            26.12   

                      hist_rcp85_p50  t2050_rcp85_p50  t2080_rcp85_p50  \
lat       lon                                                            
40.041039 -89.045212            7.11             4.99             3.26   
          -88.961884            7.27             5.09             3.22   
          -88.878548            7.34             5.14             3.24   
          -88.795219            7.33             5.12             3.26   
          -88.711891            7.30             5.06             3.23   
...                              ...              ...              ...   
66.623306 -61.629639           33.69            38.99            40.57   
          -61.546310           34.92            40.78            42.46   
          -61.462978           34.01            39.68            41.42   
          -61.379650           33.75            39.19            40.62   
          -61.296322           30.21            34.62            36.26   

                      hist_rcp85_p90  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                           
40.041039 -89.045212            9.54             5.54             4.04  
          -88.961884            9.70             5.63             4.17  
          -88.878548            9.81             5.71             4.07  
          -88.795219            9.70             5.68             4.03  
          -88.711891            9.72             5.72             4.08  
...                              ...              ...              ...  
66.623306 -61.629639           36.49            39.84            44.01  
          -61.546310           38.05            41.65            46.12  
          -61.462978           36.94            40.74            44.72  
          -61.379650           36.50            40.23            44.18  
          -61.296322           32.80            36.00            39.44  

[76206 rows x 9 columns]

# Merge df45 and 85

In [26]:
dftg = pd.merge(df45f, df85f, on=["lat","lon"])
dftg.head()

hist_rcp45_p10  t2050_rcp45_p10  t2080_rcp45_p10  \
lat       lon                                                            
40.041039 -89.045212            5.78             3.29             3.32   
          -88.961884            5.93             3.35             3.36   
          -88.878548            6.04             3.37             3.39   
          -88.795219            6.04             3.33             3.36   
          -88.711891            6.06             3.27             3.30   

                      hist_rcp45_p50  t2050_rcp45_p50  t2080_rcp45_p50  \
lat       lon                                                            
40.041039 -89.045212            6.94             5.44             4.68   
          -88.961884            7.11             5.56             4.72   
          -88.878548            7.12             5.64             4.76   
          -88.795219            7.14             5.66             4.78   
          -88.711891            7.11             5.67             4.68   

                      hist_rcp45_p90  t2050_rcp45_p90  t2080_rcp45_p90  \
lat       lon                                                            
40.041039 -89.045212            8.95             7.10             6.45   
          -88.961884            9.16             7.28             6.55   
          -88.878548            9.24             7.29             6.63   
          -88.795219            9.16             7.26             6.65   
          -88.711891            9.19             7.17             6.65   

                      hist_rcp85_p10  t2050_rcp85_p10  t2080_rcp85_p10  \
lat       lon                                                            
40.041039 -89.045212            6.03             2.77             1.60   
          -88.961884            6.23             2.85             1.61   
          -88.878548            6.34             2.90             1.60   
          -88.795219            6.44             2.91             1.66   
          -88.711891            6.37             2.85             1.59   

                      hist_rcp85_p50  t2050_rcp85_p50  t2080_rcp85_p50  \
lat       lon                                                            
40.041039 -89.045212            7.11             4.99             3.26   
          -88.961884            7.27             5.09             3.22   
          -88.878548            7.34             5.14             3.24   
          -88.795219            7.33             5.12             3.26   
          -88.711891            7.30             5.06             3.23   

                      hist_rcp85_p90  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                           
40.041039 -89.045212            9.54             5.54             4.04  
          -88.961884            9.70             5.63             4.17  
          -88.878548            9.81             5.71             4.07  
          -88.795219            9.70             5.68             4.03  
          -88.711891            9.72             5.72             4.08

# Extract hist

In [27]:
dftg1 = dftg.drop(columns=["hist_rcp45_p10","hist_rcp45_p50", "hist_rcp45_p90", "hist_rcp85_p10", "hist_rcp85_p50", "hist_rcp85_p90"])
dftgh = dftg[["hist_rcp45_p50", "hist_rcp85_p50"]]

In [28]:
dftg1

t2050_rcp45_p10  t2080_rcp45_p10  t2050_rcp45_p50  \
lat       lon                                                             
40.041039 -89.045212             3.29             3.32             5.44   
          -88.961884             3.35             3.36             5.56   
          -88.878548             3.37             3.39             5.64   
          -88.795219             3.33             3.36             5.66   
          -88.711891             3.27             3.30             5.67   
...                               ...              ...              ...   
66.623306 -61.629639            35.02            34.88            36.67   
          -61.546310            36.79            36.48            38.72   
          -61.462978            35.69            35.50            37.45   
          -61.379650            35.15            35.20            37.15   
          -61.296322            31.37            30.84            33.28   

                      t2080_rcp45_p50  t2050_rcp45_p90  t2080_rcp45_p90  \
lat       lon                                                             
40.041039 -89.045212             4.68             7.10             6.45   
          -88.961884             4.72             7.28             6.55   
          -88.878548             4.76             7.29             6.63   
          -88.795219             4.78             7.26             6.65   
          -88.711891             4.68             7.17             6.65   
...                               ...              ...              ...   
66.623306 -61.629639            38.81            41.11            42.72   
          -61.546310            40.47            43.01            44.13   
          -61.462978            39.31            41.86            43.32   
          -61.379650            38.93            41.25            42.71   
          -61.296322            34.90            37.24            38.55   

                      t2050_rcp85_p10  t2080_rcp85_p10  t2050_rcp85_p50  \
lat       lon                                                             
40.041039 -89.045212             2.77             1.60             4.99   
          -88.961884             2.85             1.61             5.09   
          -88.878548             2.90             1.60             5.14   
          -88.795219             2.91             1.66             5.12   
          -88.711891             2.85             1.59             5.06   
...                               ...              ...              ...   
66.623306 -61.629639            32.11            29.49            38.99   
          -61.546310            34.50            31.98            40.78   
          -61.462978            33.64            31.19            39.68   
          -61.379650            33.31            30.88            39.19   
          -61.296322            29.48            26.12            34.62   

                      t2080_rcp85_p50  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                            
40.041039 -89.045212             3.26             5.54             4.04  
          -88.961884             3.22             5.63             4.17  
          -88.878548             3.24             5.71             4.07  
          -88.795219             3.26             5.68             4.03  
          -88.711891             3.23             5.72             4.08  
...                               ...              ...              ...  
66.623306 -61.629639            40.57            39.84            44.01  
          -61.546310            42.46            41.65            46.12  
          -61.462978            41.42            40.74            44.72  
          -61.379650            40.62            40.23            44.18  
          -61.296322            36.26            36.00            39.44  

[76206 rows x 12 columns]

# Merge both hist_50 and get the mean

In [29]:
dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)
dftgh2 = dftgh.reset_index()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)



In [30]:
dftgh3 = dftgh2[["lat", "lon", "hist_p50"]]
dftgh3

,lat,lon,hist_p50
0,40.041039,-89.045212,7.02
1,40.041039,-88.961884,7.19
2,40.041039,-88.878548,7.23
3,40.041039,-88.795219,7.24
4,40.041039,-88.711891,7.20
...,...,...,...
76201,66.623306,-61.629639,33.84
76202,66.623306,-61.546310,35.11
76203,66.623306,-61.462978,34.19
76204,66.623306,-61.379650,33.83


# Merge means with the 45 and 85 DF

In [31]:
dftg_all = pd.merge(dftgh3, dftg1, on=["lat","lon"])

In [32]:
dftg_all

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,7.02,3.29,3.32,5.44,4.68,7.10,6.45,2.77,1.60,4.99,3.26,5.54,4.04
1,40.041039,-88.961884,7.19,3.35,3.36,5.56,4.72,7.28,6.55,2.85,1.61,5.09,3.22,5.63,4.17
2,40.041039,-88.878548,7.23,3.37,3.39,5.64,4.76,7.29,6.63,2.90,1.60,5.14,3.24,5.71,4.07
3,40.041039,-88.795219,7.24,3.33,3.36,5.66,4.78,7.26,6.65,2.91,1.66,5.12,3.26,5.68,4.03
4,40.041039,-88.711891,7.20,3.27,3.30,5.67,4.68,7.17,6.65,2.85,1.59,5.06,3.23,5.72,4.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,33.84,35.02,34.88,36.67,38.81,41.11,42.72,32.11,29.49,38.99,40.57,39.84,44.01
76202,66.623306,-61.546310,35.11,36.79,36.48,38.72,40.47,43.01,44.13,34.50,31.98,40.78,42.46,41.65,46.12
76203,66.623306,-61.462978,34.19,35.69,35.50,37.45,39.31,41.86,43.32,33.64,31.19,39.68,41.42,40.74,44.72
76204,66.623306,-61.379650,33.83,35.15,35.20,37.15,38.93,41.25,42.71,33.31,30.88,39.19,40.62,40.23,44.18


In [33]:
dftg_all.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_annual.feather")

In [34]:
#dftg_all = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_annual.feather")

In [35]:
dftg_all 

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,7.02,3.29,3.32,5.44,4.68,7.10,6.45,2.77,1.60,4.99,3.26,5.54,4.04
1,40.041039,-88.961884,7.19,3.35,3.36,5.56,4.72,7.28,6.55,2.85,1.61,5.09,3.22,5.63,4.17
2,40.041039,-88.878548,7.23,3.37,3.39,5.64,4.76,7.29,6.63,2.90,1.60,5.14,3.24,5.71,4.07
3,40.041039,-88.795219,7.24,3.33,3.36,5.66,4.78,7.26,6.65,2.91,1.66,5.12,3.26,5.68,4.03
4,40.041039,-88.711891,7.20,3.27,3.30,5.67,4.68,7.17,6.65,2.85,1.59,5.06,3.23,5.72,4.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,33.84,35.02,34.88,36.67,38.81,41.11,42.72,32.11,29.49,38.99,40.57,39.84,44.01
76202,66.623306,-61.546310,35.11,36.79,36.48,38.72,40.47,43.01,44.13,34.50,31.98,40.78,42.46,41.65,46.12
76203,66.623306,-61.462978,34.19,35.69,35.50,37.45,39.31,41.86,43.32,33.64,31.19,39.68,41.42,40.74,44.72
76204,66.623306,-61.379650,33.83,35.15,35.20,37.15,38.93,41.25,42.71,33.31,30.88,39.19,40.62,40.23,44.18


# Merge complete DF with Polygons for each spatial scale

## Dictionary - regioins: column name, short name

In [36]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
              "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"), 
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [37]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [38]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "à": "a", "è": "e", "ô" : "o", "ç":"c", "É": "E", "È": "E", "Î": "i", "Ç":"C" } 

In [39]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftg_all, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_annual.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_annual.json", driver="GeoJSON")

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_solidprcptot_annual.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
